In [1]:
# Keras
from keras.layers import Dense, Dropout
from keras import Input, Sequential

# Sklearn
from sklearn.model_selection import train_test_split

# Mediapipe Processor
from MediapipeProcessor import MediaPipeProcessor

# Misc
import pandas as pd
import cv2
import pickle
import numpy as np

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# Building the model

## Architecture
This model is a simple neural network (NN) with the following architecture:
- Dense 33 x 3 units (number of landmarks times numnber of dimensions), relu activation
- Dropout (rate TBD)
- Dense 1 unit, sigmoid activation


In [53]:
n_landmark = 33
n_dim = 3
n_params = n_landmark * n_dim

In [67]:
model = Sequential(
    [
        Input(shape=(n_params,)),
        Dense(units=n_params, activation="relu"),
        Dropout(rate=0.1),
        Dense(units=1, activation='sigmoid')
    ]
)

## Preparing dataset

In [2]:
processor = MediaPipeProcessor()

I0000 00:00:1765894313.626818   53378 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1765894313.630629   53434 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: Mesa Intel(R) HD Graphics 530 (SKL GT2)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1765894313.723407   53428 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765894313.785124   53422 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765894313.791352   53423 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765894313.793507   53427 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765894313.798119   53429 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765894313.806499   53

In [8]:
processor.get_landmarks_from_folder("../data/img/bas", "../data/landmarks/bas")
processor.get_landmarks_from_folder("../data/img/haut", "../data/landmarks/haut")

In [32]:
data_bas = pd.read_pickle("../data/landmarks/bas/landmarks.pickle")
data_bas = np.array([processor.landmarks_to_array(data_bas["landmarks"][i]).flatten() for i in range(len(data_bas["landmarks"]))])

In [33]:
data_haut = pd.read_pickle("../data/landmarks/haut/landmarks.pickle")
data_haut = np.array([processor.landmarks_to_array(data_haut["landmarks"][i]).flatten() for i in range(len(data_haut["landmarks"]))])

In [49]:
X = np.concatenate([data_bas, data_haut])
y = np.concatenate([np.repeat(1, len(data_bas)), np.repeat(0, len(data_haut))])


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=37)

## Training the model

In [68]:
model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=["accuracy"]
)

In [69]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=200)

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.8333 - loss: 0.2121 - val_accuracy: 0.7500 - val_loss: 0.1847
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7396 - loss: 0.1841 - val_accuracy: 0.7500 - val_loss: 0.1646
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7292 - loss: 0.1671 - val_accuracy: 0.7500 - val_loss: 0.1520
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7396 - loss: 0.1531 - val_accuracy: 0.7500 - val_loss: 0.1421
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7708 - loss: 0.1422 - val_accuracy: 0.7917 - val_loss: 0.1331
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8438 - loss: 0.1304 - val_accuracy: 0.8333 - val_loss: 0.1251
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8854 - loss: 0.1243 - val_accuracy: 0.8333 - val_loss: 0.1182
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9062 - loss: 0.1134 - val_accuracy: 0.8750 - val_loss:

In [75]:
mse, acc = model.evaluate(X_test, y_test)
print(f"MSE: {mse:.2f}\nAccuracy: {100*acc:.1f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9024 - loss: 0.0548
MSE: 0.05
Accuracy: 90.2%


In [76]:
model.save("model.keras")